In [114]:
df.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN


# Movies Dataset

In [115]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bharatnatrayn/movies-dataset-for-feature-extracion-prediction")

print("Path to dataset:", path)

Using Colab cache for faster access to the 'movies-dataset-for-feature-extracion-prediction' dataset.
Path to dataset: /kaggle/input/movies-dataset-for-feature-extracion-prediction


In [116]:
import pandas as pd

In [154]:
df = pd.read_csv(f"{path}/movies.csv")

### Initial Inspection

In [155]:
df.shape

(9999, 9)

In [156]:
df.columns

Index(['MOVIES', 'YEAR', 'GENRE', 'RATING', 'ONE-LINE', 'STARS', 'VOTES',
       'RunTime', 'Gross'],
      dtype='object')

In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


In [158]:
df.describe()

,RATING,RunTime
count,8179.000000,7041.000000
mean,6.921176,68.688539
std,1.220232,47.258056
min,1.100000,1.000000
25%,6.200000,36.000000
50%,7.100000,60.000000
75%,7.800000,95.000000
max,9.900000,853.000000


In [159]:
df.head(3)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN


## Improper Formatting Issues
Clear \n

In [160]:
df['GENRE'][0]

'\nAction, Horror, Thriller            '

In [161]:
df['STARS'][0]

'\n    Director:\nPeter Thorwarth\n| \n    Stars:\nPeri Baumeister, \nCarl Anton Koch, \nAlexander Scheer, \nKais Setti\n'

In [162]:
df['ONE-LINE'][0]

'\nA woman with a mysterious illness is forced into action when a group of terrorists attempt to hijack a transatlantic overnight flight.'

### Cleaning

In [163]:
df_formatting = df.copy()
cols = ['GENRE','STARS','ONE-LINE']
for col in cols:
    df_formatting[col] = df_formatting[col].str.replace('\n','').str.strip()

In [164]:
df_formatting['GENRE'][0]

'Action, Horror, Thriller'

In [165]:
df_formatting['STARS'][0]

'Director:Peter Thorwarth|     Stars:Peri Baumeister, Carl Anton Koch, Alexander Scheer, Kais Setti'

In [166]:
df_formatting['ONE-LINE'][0]

'A woman with a mysterious illness is forced into action when a group of terrorists attempt to hijack a transatlantic overnight flight.'

## Inconsistent Formatting
Break into Columns

In [167]:
df['YEAR'][0:3]

,YEAR
0,(2021)
1,(2021– )
2,(2010–2022)


Removing '(' & ')'

In [168]:
df_year = df_formatting.copy()
df_year['YEAR'] = df_year['YEAR'].str.replace('(', '').str.replace(')', '').str.strip()
df_year['YEAR'][0:3]

,YEAR
0,2021
1,2021–
2,2010–2022


Is movie/series

In [169]:
df_year[['Start_Year', 'End_Year']] = (df_year['YEAR'].str.split('–', expand=True))


In [170]:
df_year['Start_Year'] = pd.to_numeric(df_year['Start_Year'], errors='coerce')
df_year['End_Year'] = pd.to_numeric(df_year['End_Year'], errors='coerce')

Movie/Series

In [171]:
df_year['Is_Series'] = df_year['YEAR'].str.contains('–', na=False)

df_year['Is_Ongoing'] = (df_year['Is_Series'] & df_year['End_Year'].isna())

In [172]:
df_year.drop(columns=['YEAR'], inplace=True)
df_year.head(3)


,MOVIES,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross,Start_Year,End_Year,Is_Series,Is_Ongoing
0,Blood Red Sky,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth| Stars:Peri Baume...,"21,062",121.0,NaN,2021.0,NaN,False,False
1,Masters of the Universe: Revelation,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,"Stars:Chris Wood, Sarah Michelle Gellar, Lena ...","17,870",25.0,NaN,2021.0,NaN,True,True
2,The Walking Dead,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman Reedus, Melissa M...","885,805",44.0,NaN,2010.0,2022.0,True,False


### Missing Values in Year
(NOt DONE YET)

- start date nan -> avg of same end date's start date by movie/series

In [173]:
print(df_year['Start_Year'].isnull().sum())
print(df_year['End_Year'].isnull().sum())

1692
8611


In [174]:
df_year['Is_Series'].value_counts()

,count
Is_Series,
False,5431
True,4568


## Non-atomic / Multi-valued Attributes

In [175]:
df_year['GENRE'][0:3]

,GENRE
0,"Action, Horror, Thriller"
1,"Animation, Action, Adventure"
2,"Drama, Horror, Thriller"


In [176]:
df_genre = df_year.copy()
df_genre['GENRE'] = df_genre['GENRE'].str.strip()
df_genre['GENRE'] = df_genre['GENRE'].str.split(', ')

In [177]:
df_genre['Genre_Count'] = df_genre['GENRE'].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)

df_genre['Genre_Count'].value_counts()

,count
Genre_Count,
3,5141
1,2582
2,2196
0,80


## Missing Values

In [178]:
df_genre.isnull().sum()

,0
MOVIES,0
GENRE,80
RATING,1820
ONE-LINE,0
STARS,0
VOTES,1820
RunTime,2958
Gross,9539
Start_Year,1692
End_Year,8611


- Gross: Drop
- Unvoted: Drop -> Vote/Rating

In [179]:
df_test = df_genre.copy()
df_test = df_test.dropna(subset=['VOTES'])
df_test.isnull().sum()

,0
MOVIES,0
GENRE,11
RATING,0
ONE-LINE,0
STARS,0
VOTES,0
RunTime,1393
Gross,7719
Start_Year,876
End_Year,6829


In [192]:
df_test = df_test.drop(columns=['Gross'])
df_test.isnull().sum()

,0
MOVIES,0
GENRE,11
RATING,0
ONE-LINE,0
STARS,0
VOTES,0
RunTime,1393
Start_Year,876
End_Year,6829
Is_Series,0


### Missing Value for Runtime

In [195]:
df_test.groupby('Is_Series')['RunTime'].mean()

,RunTime
Is_Series,
False,90.492119
True,39.310864


In [197]:
df_test.loc[(df_test['Is_Series'] == False) & (df_test['RunTime'].isna()), 'RunTime'] = 90.492119
df_test.loc[(df_test['Is_Series'] == True) & (df_test['RunTime'].isna()), 'RunTime'] = 39.310864

In [198]:
df_test.groupby('Is_Series')['RunTime'].mean()

,RunTime
Is_Series,
False,90.492119
True,39.310864


In [199]:
df_test.shape

(8179, 12)

## Feature Extraction

### Drop less useful

In [200]:
df_feature = df_test.copy()

In [201]:
df_feature.head(3)

,MOVIES,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Start_Year,End_Year,Is_Series,Is_Ongoing,Genre_Count
0,Blood Red Sky,"[Action, Horror, Thriller]",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth| Stars:Peri Baume...,"21,062",121.0,2021.0,NaN,False,False,3
1,Masters of the Universe: Revelation,"[Animation, Action, Adventure]",5.0,The war for Eternia begins again in what may b...,"Stars:Chris Wood, Sarah Michelle Gellar, Lena ...","17,870",25.0,2021.0,NaN,True,True,3
2,The Walking Dead,"[Drama, Horror, Thriller]",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman Reedus, Melissa M...","885,805",44.0,2010.0,2022.0,True,False,3


In [202]:
df_feature = df_feature.drop(columns=['ONE-LINE', 'STARS'])

In [203]:
df_feature.head()

,MOVIES,GENRE,RATING,VOTES,RunTime,Start_Year,End_Year,Is_Series,Is_Ongoing,Genre_Count
0,Blood Red Sky,"[Action, Horror, Thriller]",6.1,"21,062",121.0,2021.0,NaN,False,False,3
1,Masters of the Universe: Revelation,"[Animation, Action, Adventure]",5.0,"17,870",25.0,2021.0,NaN,True,True,3
2,The Walking Dead,"[Drama, Horror, Thriller]",8.2,"885,805",44.0,2010.0,2022.0,True,False,3
3,Rick and Morty,"[Animation, Adventure, Comedy]",9.2,"414,849",23.0,2013.0,NaN,True,True,3
5,Outer Banks,"[Action, Crime, Drama]",7.6,"25,858",50.0,2020.0,NaN,True,True,3


## Final Analysis

In [204]:
df_feature.shape

(8179, 10)

In [205]:
df_feature.isnull().sum()

,0
MOVIES,0
GENRE,11
RATING,0
VOTES,0
RunTime,0
Start_Year,876
End_Year,6829
Is_Series,0
Is_Ongoing,0
Genre_Count,0
